In [1]:
#nin块的设计
import torch 
from torch import nn
from d2l import torch as d2l

def nin_block(in_channels,out_chnnels,kernel_size,strides,padding):
    return nn.Sequential(
        nn.Conv2d(in_channels,out_chnnels,kernel_size,strides,padding),
        nn.ReLU(),
        nn.Conv2d(out_chnnels,out_chnnels,kernel_size=1),nn.ReLU(),
        nn.ReLU(),
        nn.Conv2d(out_chnnels,out_chnnels,kernel_size=1),nn.ReLU(),
        nn.ReLU()
    )

In [3]:
net=nn.Sequential(
    #初始输入的图像为(1,224,224)
    #这一层之后输出的为(96,54,54)
    nin_block(1,96,kernel_size=11,strides=4,padding=0),
    #这一层之后输出为(96,26,26)
    nn.MaxPool2d(kernel_size=3,stride=2),
    #这一层之后输出为(256,26,26)
    nin_block(96,256,kernel_size=5,strides=1,padding=2),
    #这一层输出之后为(256,12,12)
    nn.MaxPool2d(kernel_size=3,stride=2),
    #这一层输出之后为(384,12,12)
    nin_block(256,384,kernel_size=3,strides=1,padding=1),
    #这一层之后输出为(384,5,5)
    nn.MaxPool2d(kernel_size=3,stride=2),
    nn.Dropout(0.5),
    #标签类别数是10 所以最后输出的通道数要为10
    #这一层输出为(10,5,5)
    nin_block(384,10,kernel_size=3,strides=1,padding=1),
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(),
)

In [4]:
X=torch.rand(1,1,224,224)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [ ]:
#开始训练模型
lr,num_epochs,batch_size=0.1,10,128
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=224)
d2l.train_ch6(net,train_iter,test_iter,num_epochs,lr,d2l.try_gpu())